In [ ]:
import skmine
print("This tutorial was tested with the following version of skmine :", skmine.__version__)

# Table of Contents

* [Chapter 1 Discretizing Iris Dataset](#chapter1)
    * [Section 1 Original Dataset](#section_1)
    * [Section 1](#Section_2)
        * [Section 1.2.1](#section_1_2_1)
        * [Section 1.2.2](#section_1_2_2)
        * [Section 1.2.3](#section_1_2_3)
* [Chapter 2](#chapter2)
    * [Section 2.1](#section_2_1)

In [1]:
import skmine
print("This tutorial was tested with the following version of skmine :", skmine.__version__)

This tutorial was tested with the following version of skmine : 0.0.9


# Chapter 1 Discretizing Iris Dataset <a class="anchor" id="chapter1"></a>
 ## Section 1: Original dataset <a class="anchor" id="section_1"></a>

In [58]:
from sklearn.datasets import load_iris
import pandas as pd

iris  = load_iris(as_frame=True)
 # pd.DataFrame(iris.data, columns=iris.feature_names)
print("Target: \n", *list(enumerate(iris.target_names)),sep='\n')
iris.frame


Target: 

(0, 'setosa')
(1, 'versicolor')
(2, 'virginica')


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [59]:
import plotly.express as px
import pandas as pd
from sklearn import datasets

# Charger les données IRIS
iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['species'] = iris.target

# Créer un graphique 3D interactif
fig = px.scatter_3d(iris_df, x='sepal length (cm)', y='sepal width (cm)', z='petal length (cm)',
              color='species', labels={'species':'species'},
              title="Iris Dataset in 3D",)
fig.update_traces(marker_size = 3)
fig.show()


 ## Section 2: Discretize <a class="anchor" id="section_2"></a>